In [1]:
from huggingface_hub import login
from transformers import AutoTokenizer, AutoModelForCausalLM, XLMRobertaTokenizerFast, XLMRobertaXLForCausalLM
import torch
import numpy as np

/home/wolfingten/.python_environments/ML/lib/python3.11/site-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/home/wolfingten/.python_environments/ML/lib/python3.11/site-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/home/wolfingten/.python_environments/ML/lib/python3.11/site-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


In [2]:
tokenizer = XLMRobertaTokenizerFast.from_pretrained("xlm-roberta-base")
model = XLMRobertaXLForCausalLM.from_pretrained("xlm-roberta-base")

You are using a model of type xlm-roberta to instantiate a model of type xlm-roberta-xl. This is not supported for all configurations of models and can yield errors.
If you want to use `RobertaLMHeadModel` as a standalone, add `is_decoder=True.`
Some weights of XLMRobertaXLForCausalLM were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['roberta.encoder.layer.7.attention.self_attn_layer_norm.bias', 'roberta.encoder.layer.2.LayerNorm.weight', 'roberta.encoder.layer.1.attention.self_attn_layer_norm.bias', 'roberta.encoder.layer.10.LayerNorm.weight', 'roberta.encoder.layer.3.LayerNorm.weight', 'roberta.encoder.layer.6.attention.self_attn_layer_norm.weight', 'roberta.encoder.layer.6.LayerNorm.bias', 'roberta.encoder.layer.4.attention.self_attn_layer_norm.weight', 'roberta.encoder.layer.11.attention.self_attn_layer_norm.bias', 'roberta.encoder.layer.5.attention.self_attn_layer_norm.bias', 'roberta.encoder.layer.2.LayerNorm.bias', 'roberta.encoder.la

In [32]:
tokens = tokenizer("I am bald unbelievable.", return_tensors="pt")
with torch.no_grad():
    outputs = model(**tokens)


In [20]:
output_ids = tokens["input_ids"].squeeze(0)[1:-1]
index = torch.arange(0, output_ids.shape[0])
surp = -1 * torch.log2(torch.nn.functional.softmax(outputs.logits, dim=-1).squeeze(0)[index, output_ids])
surp

In [26]:
ids = np.array(ids)
surprisal_values = surp.numpy()

word_surprisal_sum = np.bincount(ids, weights=surprisal_values)
word_counts = np.bincount(ids)
word_surprisal_avg = word_surprisal_sum / word_counts
word_surprisal_avg

In [37]:
def calc_word_surprisal(tokens, output):
    out_ids = tokens["input_ids"].squeeze(0)[1:-1]
    mask = torch.arange(0, out_ids.shape[0])
    subword_surp = -1 * torch.log2(torch.nn.functional.softmax(outputs.logits, dim=-1).squeeze(0)[index, output_ids])

    ids = np.array(tokens.word_ids()[1:-1])
    word_surp_sum = np.bincount(ids, weights=subword_surp.numpy())
    word_cnts = np.bincount(ids)
    word_surp_avg = word_surp_sum / word_cnts
    return word_surprisal_avg

calc_word_surprisal(tokens, outputs)